# Project Description
### *Trypanosoma brucei* performs RNA editing on several mitochondrially expressed gene transcripts. This editing takes place in the form of uridine insertions or deletions. In a population of transcripts generated from a single gene, many different forms can exist, with different amounts of Us at different places. The goal of this project is to align multiple sequences of the same gene transcript at different stages of editing.

### This tool is applicable to sequences of edited mRNAs generated in all kinetoplastid protozoa.  It is not designed for other types of RNA editing, such as A to I, but could possibly work those, as the basis of this program is a sequence alignment program.  However, this would be quite unnecessary as the special treatment of Us in this program is a waste of time for an A to I edited transcript.  

### So far, I have only tested this program on 31 total sequences.  I will be testing it soon on larger data sets, but probably no larger than 100 sequences, as I do not have this data currently.  I have tested it on two genes so far, and plan to test it on more data, as I assemble it.  There are only 12 edited genes in *T. brucei*.

## Sequence Class
### To perform this task, we need to generate several pieces of information from each sequence, such as the name of the sequence, the sequence itself, the sequence with Us removed and the list of the number of Us at each potential U site.  To tie these pieces together, I made Sequence class.  This class is only able to extract the information if it is given in fasta format.  It also converts the sequences to uppercase, removes asterisks and Ts.  

In [92]:
class Sequence:
    def __init__(self,string):
        self.content=string
        #remove *s, us, and Ts
        self.content=self.content.replace("*","")
        self.content=self.content.replace("u","U")
        self.content=self.content.replace("T","U")
        self.content=self.content.replace("a","A")
        self.content=self.content.replace("g","G")
        self.content=self.content.replace("c","C")

        #count the length of the name
        self.titleLength=0
        while self.content[self.titleLength]!=("\n"):
            self.titleLength=self.titleLength+1
        
        #make the title
        self.title=""
        for i in range(self.titleLength-1):
            self.title=self.title+self.content[i+1]#this excludes the ">" at the beginning of each line
        
        #extract the sequence by looking after the title
        self.sequence=self.content[self.titleLength+1]
        for i in range(self.titleLength+2,len(self.content)):
            self.sequence=self.sequence+self.content[i]
        self.sequence=self.sequence.replace("\n","")#remove linebreaks
        
        #make the noU sequence and the U list
        self.uList=list()
        uCounter=0 #counts the number of Us at any given position
        seqPos=0 #is the placekeeper in the sequence
        self.noUs=""
        #initialize the uList
        if self.sequence[0]=="U":
            uCounter=1
            seqPos=1
            #count how many consecutive Us are at this position
            while self.sequence[uCounter]=="U":
                uCounter=uCounter+1
                seqPos=seqPos+1
            self.uList.append(uCounter)
        else:
            self.uList.append(0)
        #add each non-U nucleotide to the noUs string and count how many Us come after each nucleotide
        while seqPos<len(self.sequence):
            self.noUs=self.noUs+self.sequence[seqPos]
            seqPos=seqPos+1
            uCounter=0
            if seqPos<len(self.sequence):
                if self.sequence[seqPos]=="U":
                    uCounter=1
                    seqPos=seqPos+1
                    while seqPos<len(self.sequence) and self.sequence[seqPos]=="U":
                        uCounter=uCounter+1
                        seqPos=seqPos+1
                    self.uList.append(uCounter)
                else:
                    self.uList.append(0)
        #for some reason, it wasn't adding the U at the end every time, so I added these two lines
        if self.sequence[seqPos-1]!="U":
            self.uList.append(0)
            

# extractSequences function
### This function serves to open a text file with a list of sequences in the fasta format and extract each sequence.  It saves all of these in a list and saves each sequence as the class, Sequence (see above), in a new list, allSeq, which is returned.  
### In the fasta format, each sequence is denoted by a ">" followed by the name of the sequence, with the actual sequence on the following line.  This function will fail if sequences are not input in this method.

In [93]:
def extractSequences(filename):
    F=open(filename)
    content=F.readlines()
    F.close()
    sequences=list() #dummy list to hold raw sequence information
    for lines in range(len(content)):
        #find the start of the sequence
        if content[lines][0]==">":
            sequence=""
            sequence=sequence+content[lines]+content[lines+1]
            sequences.append(sequence)
    allSeq=list()
    #convert all raw sequences to the Sequence class in a new list
    for i in range(len(sequences)):
        seqi=Sequence(sequences[i])
        allSeq.append(seqi)
    return allSeq

In [94]:
ND3shortSeq=extractSequences("ND3align.fa")
print "There are {0} sequences in this file.".format(len(ND3shortSeq))

There are 10 sequences in this file.


# scoreMatrix function
### This function serves to generate a score matrix for two sequences.  It does this by creating a table that is one column longer than the first sequence and one row longer than than the second sequence.  This first row and column are filled with zeros, the second row at the second position, is then scored by evaluating the first nucleotide pair.  Scores for all other cells are filled in using the scores of cells left, above and above and left to determine which will yield the highest value of the three.  If the nucleotides match, the above and left cell score recieves a matching bonus, but if they do not it receives a mismatch penalty.  Above scores and left scores receive a gap penalty.  

### The gap penalty, mismatch penalty and match bonus were not chosen at random.  These were selected based on which set of numbers generated good outputs.  

In [95]:
def scoreMatrix(seq1,seq2):
    #initialize the matrix
    matrix=list()
    for i in range(len(seq1)+1):
        matrix.append(list())
        for j in range(len(seq2)+1):
            matrix[i].append(0.0)

    #fill it in with scores
    k=1 #keep track of position left to right
    l=1 #keep track of position up and down
    gapPenalty=-1
    mismatchPenalty=0
    matchScore=1
    while l<(len(seq2)+1):
        while k<(len(seq1)+1):
            score1=matrix[k][l-1]+gapPenalty#above
            score2=matrix[k-1][l]+gapPenalty#left
            score3=matrix[k-1][l-1]+mismatchPenalty#above and to the left
            if seq1[k-1]==seq2[l-1]:
                score3=score3+(matchScore-mismatchPenalty)
            matrix[k][l]=max(score1,score2,score3)
            k=k+1
        l=l+1
        k=1
    return matrix

In [96]:
ND30and1matrix=scoreMatrix(ND3shortSeq[0].noUs,ND3shortSeq[1].noUs)
print len(ND3shortSeq[0].noUs)
print len(ND30and1matrix) #should be one greater
print len(ND3shortSeq[1].noUs)
print len(ND30and1matrix[0]) #should be one greater


180
181
49
50


# dashPositions function and makeAlignment function
### The dashPositions function uses the matrix previously generated and the sequences used to generate it to create two lists that will later dictate where dashes should be inserted to create the sequence alignment in makeAlignment.  This occurs by first identifiying the highest end score and tracing back through the matrix along the path with the highest scores.  The two lists record the position along each sequence as the function steps through the score matrix.  

### The makeAlignment function looks at the two lists from dashPositions and generates two aligned sequences.  It does this by reading each position in the list and its next neighbor.  If the two positions have the same number, a dash is appended to the new aligned sequence, if not, the nucleotide in the sequence is added.   This continues to the end of the sequence

In [97]:
def dashPositions(seq1,seq2,matrix):
    #Find the position with the highest final score, which is the end of the best alignment
    maxScore=-1000
    m=len(seq1)#position left to right
    n=len(seq2)#position up and down
    
    #This if statement makes it start searching for the highest score along the length of the longer of the two sequences
    if len(seq1)>len(seq2):
        for i in range(len(matrix)):
            if matrix[i][(len(matrix[i])-1)]>maxScore:
                maxScore=matrix[i][(len(matrix[i])-1)]
                m=i
                n=(len(matrix[i]))-1
        for i in range(len(matrix[0])):
            if matrix[len(matrix)-1][i]>maxScore:
                m=(len(matrix))-1
                n=i
    else:
        for i in range(len(matrix[0])):
            if matrix[len(matrix)-1][i]>maxScore:
                m=(len(matrix))-1
                n=i
        for i in range(len(matrix)):
            if matrix[i][(len(matrix[i])-1)]>maxScore:
                maxScore=matrix[i][(len(matrix[i])-1)]
                m=i
                n=(len(matrix[i]))-1
        
    
    Ms=list()
    Ns=list()
    alignmentScore=matrix[m][n]
    left=0
    up=0
    
    #count up all the ms and ns
    while (m+n)!=0:
        Ms.append(m)
        Ns.append(n)
        if m==0:
            n=n-1
        else:
            if n==0:
                m=m-1
            else:            
                score1=matrix[m][n-1]#above
                score2=matrix[m-1][n]#left
                score3=matrix[m-1][n-1]#above and to the left
                #the up and left counters prevent "stair stepping" which generates alignments that look like:
                #G-A-C-G-A
                #-G-A-C-G-A
                if score1>score2 and score1>score3 and left==0:
                    n=n-1
                    up=1
                    left=0
                else:
                    if score2>score3 and up==0:
                        m=m-1
                        left=1
                        up=0
                    else:
                        m=m-1
                        n=n-1
                        left=0
                        up=0
               
    Ms.reverse()
    Ns.reverse()
    
    #add in any missing numbers from not starting at the very ends of the sequences
    
    #these add in numbers that account for the nucleotides after the end of the best alignment
    while max(Ms)<len(seq1):
        Ms.append(max(Ms)+1)
    while max(Ns)<len(seq2):
        Ns.append(max(Ns)+1)
    #these add in numbers that generate dashes that make the two sequences the same length
    while len(Ms)<len(Ns):
        Ms.append(max(Ms))
    while len(Ns)<len(Ms):
        Ns.append(max(Ns))

    
    return Ms,Ns,alignmentScore

In [98]:
ND3seq0Pos,ND3seq1Pos,ND30and1Score=dashPositions(ND3shortSeq[0].noUs,ND3shortSeq[1].noUs,ND30and1matrix)
print len(ND3seq0Pos)
print len(ND3seq1Pos)#should be equal
print ND30and1Score

180
180
48.0


In [99]:
def makeAlignment(seq1,seq2,Ms,Ns):
    
    #start seq1align with either dashes or the first nucleotide
    y=0
    seq1align=""
    while Ms[y]==0:
        seq1align=seq1align+"-"
        y=y+1
    seq1align=seq1align+seq1[0]

    #start seq2align with either dashes or the first nucleotide
    z=0
    seq2align=""
    
    while Ns[z]==0:
        seq2align=seq2align+"-"
        z=z+1
    seq2align=seq2align+seq2[0]    
    
    #Fill in seq1align
    o=0 #seq1 position counter 
    for i in range(y,len(Ms)-1):
        if Ms[i]==Ms[i+1]:
            seq1align=seq1align+"-"
        else:
            seq1align=seq1align+seq1[o+1]
            o=o+1
    #Fill in seq2align
    p=0 #seq2 position counter 
    for i in range(z,len(Ns)-1):
        if Ns[i]==Ns[i+1]:
            seq2align=seq2align+"-"
        else:
            seq2align=seq2align+seq2[p+1]
            p=p+1


    return seq1align,seq2align

In [100]:
ND3align0,ND3align1=makeAlignment(ND3shortSeq[0].noUs,ND3shortSeq[1].noUs,ND3seq0Pos,ND3seq1Pos)
print len(ND3align0)
print len(ND3align1)#should be equal
print ND3align0
print ND3align1

180
180
CAAAAAACCCGCCACAGGACAAAAGGAAGGGAAAAAGGAGAGAGCAGAGCGGGGGGCGGGGGAGAAGAACACGGGGAACAGGAAGGAGGGGAGAAACCAAGGGGGGGAAGGGAGAGGGGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAACAAAAAAAAAAAAAAA
-------------------------------------------------------------------------------------------------------------------GGGGGGGG-AGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAAA---------------


# quickAlign function
### This function ties together the scoreMatrix, dashPositions, and makeAlignment functions, so you can input two sequences and return their alignment and score (or their matrix and dash position lists, for diagnostic purposes). This function does not return a perfect alignment as it does not correct for stumps at the 5' end.
### Example:
### Correct alignment:
### GGGGGGGGGAGA
### -GGGGGGGGAGA
### What is actually returned:
### GGGGGGGGGAGA
### GGGGGGGG-AGA
### This is wrong, because later, editing positions at these ends will be misaligned.  However, this is corrected in a later function.

In [101]:
def quickAlign(seq1,seq2):
    matrix=scoreMatrix(seq1,seq2)
    Ms,Ns,alignmentScore=dashPositions(seq1,seq2,matrix)
    seq1align,seq2align=makeAlignment(seq1,seq2,Ms,Ns)
    return seq1align,seq2align,alignmentScore,#matrix,Ms,Ns

In [102]:
ND3quick0,ND3quick1,ND3quickScore=quickAlign(ND3shortSeq[0].noUs,ND3shortSeq[1].noUs)
print ND3quick0
print ND3quick1
print ND3quickScore

CAAAAAACCCGCCACAGGACAAAAGGAAGGGAAAAAGGAGAGAGCAGAGCGGGGGGCGGGGGAGAAGAACACGGGGAACAGGAAGGAGGGGAGAAACCAAGGGGGGGAAGGGAGAGGGGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAACAAAAAAAAAAAAAAA
-------------------------------------------------------------------------------------------------------------------GGGGGGGG-AGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAAA---------------
48.0


# findBest function
### The findBest function looks at a group of sequences, finds the best aligning pair and aligns them.  Then it stores their index numbers in a second list called bestList.  If this list does not exist, it creates a new one, and if it does exist it adds new numbers to the existing list.  
### Also, if a bestList does exist, it will search for the best pair that includes one sequence on the bestList and one sequence not on the bestList.  

In [103]:
def findBest(seqList,bestList):
    bestScore=-1000
    if bestList==None:
        bestList=list()
        #align all sequences
        for i in range(len(seqList)):
            for j in range(len(seqList)):
                if i!=j:
                    first,second,score=quickAlign(seqList[i].noUs,seqList[j].noUs)
                    #if this alignment is better than the current best, record this as the best
                    if score>bestScore:
                        bestScore=score
                        index1=i 
                        index2=j
                        best1=first
                        best2=second
        bestList.append(index1)
        bestList.append(index2)
    
    else:
        #look at only the sequences in the best list
        for i in bestList:
            #and compare them to the sequences not in the best list
            for j in range(len(seqList)):
                if j not in bestList:
                    first,second,score=quickAlign(seqList[i].noUs,seqList[j].noUs)
                    if score>bestScore:
                        bestScore=score
                        index1=i
                        index2=j
                        best1=first
                        best2=second
        #add the new guy to the best list
        bestList.append(index2)
        seqList[index2].noUs=best2
        
        #fix all the old best list members to match the new one
        #see how the best1 is different
        differences=list() #positions of the differences
        whatDifferences=list() #what the differences actually are
        n=0 #counter of the adjusted best1
        i=0 #counter of the original best1
        while i!=(len(seqList[index1].noUs)-1):
            if best1[n]!=seqList[index1].noUs[i]:
                differences.append(n)
                whatDifferences.append(best1[n])
                n=n+1
            else:
                n=n+1
                i=i+1
        numOfDif=len(differences)
        #this corrects for dashes added to the end of best1
        if (len(best1)-numOfDif)>len(seqList[index1].noUs):
            for i in range(len(best1)-numOfDif-len(seqList[index1].noUs)):
                differences.append(i+len(seqList[index1].noUs)+numOfDif)
                whatDifferences.append(best1[i+len(seqList[index1].noUs)+numOfDif])
        #update best1
        seqList[index1].noUs=best1
        #update all others
        if len(differences)>0:
            for j in bestList:
                if j!=index1 and j!=index2:
                    dummyseq=seqList[j].noUs
                    for k in range(len(differences)):
                        if differences[k]<len(seqList[j].noUs):
                            dummyseq=dummyseq[:differences[k]]+whatDifferences[k]+dummyseq[differences[k]:]
                        else:
                            dummyseq=dummyseq+whatDifferences[k]
                    seqList[j].noUs=dummyseq
                    
    
    #add all updated sequences to new list
    newSeqList=list()
    for i in range(len(seqList)):
        if (i==index1):
            seqList[i].noUs=best1
        else:
            if i==index2:
                seqList[i].noUs=best2
        newSeqList.append(seqList[i])    
    
    
    return newSeqList,bestList

In [104]:
ND3shortNew,ND3best=findBest(ND3shortSeq,None)
print ND3best #should have two items

[0, 3]


# stumpRepair function
### This function repairs the stump issue I previously mentioned in the quickAlign function description.  It checks to see if any sequence separated by a gap from the rest of the sequence could be aligned without the gap.  

In [105]:
def stumpRepair(seq1align,seq2align):
    gapCounter1=0 #counts how long a gap is between two pieces of sequence
    stumpCounter1=0 #counts how long a piece of sequence is before a gap
    breakPoint1=0 #records where in the sequence the gap ends
    gapCounter2=0
    stumpCounter2=0
    breakPoint2=0
    #are there any stumps?
    for i in range(len(seq1align)-1):
        #if there is a letter and we haven't counted gaps yet, count the stump
        if seq1align[i]!="-" and gapCounter1==0:
            stumpCounter1=stumpCounter1+1
        #if there is a dash and we have a counted stump, count the gap
        if seq1align[i]=="-" and stumpCounter1!=0:
            gapCounter1=gapCounter1+1
            #when we hit the next letter stop counting and record the break point
            if seq1align[i+1]!="-":
                breakPoint1=i
                break
        #repeat above for second sequence
        if seq2align[i]!="-" and gapCounter2==0:
            stumpCounter2=stumpCounter2+1
        if seq2align[i]=="-" and stumpCounter2!=0:
            gapCounter2=gapCounter2+1
            if seq2align[i+1]!="-":
                breakPoint2=i
                break
    
    #can we shift the stump?
    if breakPoint1!=0:
        canShift=0
        for j in range(stumpCounter1):
            #if the each nt of the stump matches where they would line up without the gap, canShift+1
            if seq1align[breakPoint1-gapCounter1-stumpCounter1+j+1]==seq2align[breakPoint1-stumpCounter1+j+1]:
                canShift=canShift+1
        #if they all match, then canShift will equal the length of the stump
        if canShift==stumpCounter1:
            dummy=""
            #add in alignment before the stump
            for k in range(breakPoint1-stumpCounter1-gapCounter1+1):
                dummy=dummy+seq1align[k]
            #add in appropriate number of dashes
            for l in range(gapCounter1):
                dummy=dummy+"-"
            #add in stump
            for m in range(stumpCounter1):
                dummy=dummy+seq1align[breakPoint1-stumpCounter1-gapCounter1+m+1]
            #add in the rest of the sequence
            for n in range(len(seq1align)-breakPoint1-1):
                dummy=dummy+seq1align[breakPoint1+n+1]
            seq1align=dummy
    if breakPoint2!=0:
        canShift=0
        for j in range(stumpCounter2):
            if seq2align[breakPoint2-gapCounter2-stumpCounter2+j+1]==seq1align[breakPoint2-stumpCounter2+j+1]:
                canShift=canShift+1
        if canShift==stumpCounter2:
            dummy=""
            for k in range(breakPoint2-stumpCounter2-gapCounter2+1):
                dummy=dummy+seq2align[k]
            for l in range(gapCounter2):
                dummy=dummy+"-"
            for m in range(stumpCounter2):
                dummy=dummy+seq2align[breakPoint2-stumpCounter2-gapCounter2+m+1]
            for n in range(len(seq2align)-breakPoint2-1):
                dummy=dummy+seq2align[breakPoint2+n+1]
            seq2align=dummy

    return seq1align,seq2align


In [106]:
print ND3quick0
print ND3quick1
ND3New0,ND3New1=stumpRepair(ND3quick0,ND3quick1)
print ND3New0
print ND3New1

CAAAAAACCCGCCACAGGACAAAAGGAAGGGAAAAAGGAGAGAGCAGAGCGGGGGGCGGGGGAGAAGAACACGGGGAACAGGAAGGAGGGGAGAAACCAAGGGGGGGAAGGGAGAGGGGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAACAAAAAAAAAAAAAAA
-------------------------------------------------------------------------------------------------------------------GGGGGGGG-AGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAAA---------------
CAAAAAACCCGCCACAGGACAAAAGGAAGGGAAAAAGGAGAGAGCAGAGCGGGGGGCGGGGGAGAAGAACACGGGGAACAGGAAGGAGGGGAGAAACCAAGGGGGGGAAGGGAGAGGGGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAACAAAAAAAAAAAAAAA
--------------------------------------------------------------------------------------------------------------------GGGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAAA---------------


# alignAll function
### Like quickAlign, alignAll ties together multiple functions, in this case, findBest and stump repair.  It repeats the use of findBest until all of the sequences have been incorporated to the best list, and therefore aligned.  Then it performs the stumpRepair function on all pairs of sequences to ensure that no stumps are misaligned.

In [107]:
def alignAll(seqList):
    newSeqList=list()
    #find the first best pair
    newSeqList,bestList=findBest(seqList,None)

    #match all sequences to the best sequences
    while len(newSeqList)!=len(bestList):
        newSeqList,bestList=findBest(newSeqList,bestList)
     
    #repair stumps by checking all pairs
    fixedList=list()
    for j in range(len(newSeqList)):
        newj=newSeqList[j].noUs
        for k in range(len(newSeqList)):
            if j!=k:
                newk=""
                newj,newk=stumpRepair(newj,newSeqList[k].noUs)
        fixedList.append(newj)
        seqList[j].noUs=newj

    return seqList

In [108]:
ND3alignedShort=alignAll(ND3shortSeq)
for i in range(len(ND3alignedShort)):
    print ND3alignedShort[i].title
    print ND3alignedShort[i].noUs

ND3Conv
CAAAAAACCCGCCACAGGACAAAAGGAAGGGAAAAAGGAGAGAGCAGAGCGGGGGGCGGGGGAGAAGAACACGGGGAACAGGAAGGAGGGGAGAAACCAAGGGGGGGAAGGGAGAGGGGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAACAAAAAAAAAAAAAAA
ND3AltF01
--------------------------------------------------------------------------------------------------------------------GGGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAAA---------------
ND3AltA02
-----AACCCGCCACAGGACAAAAGGAAGGGAAAAAGGAGAGGGCAGAGCGGGGGACGGGGGAGAAGAACACGGGGAACAGGAAGGAGGGGAGAAACCAAGGGAGGGAAGGGAGAGGGGGGGGG--------------------------------------------------------
ND3Alt4E01
-----AACCCGCCACAGGACAAAAGGAAGGGAAAAAGGAGAGGGCAGAGCGGGGGACGGGGGAGAAGAACACGGGGAACAGGAAGGAGGGGAGAAACCAAGGGAGGGAAGGGAGAGGGGGGGGGAGAAGGAGGGGAGGGACACG------------------------------------
ND3Alt2A03
---------------------------------------------------------------------------------------------------------------------GGGGGGGAGAAGGAGGGGAGGGACACGAAGGGGGAAAGAGAGGGAAAA---------------
ND3Alt2C02
-----AACCCGCCACAGGACAAGAGGAAGGGAAA

# updateUs function
### This function looks at an aligned sequence and identifies the locations of dashes.  Because dashes indicate that there are other potential editing sites, it then goes to the corresponding position in the u list and adds a zero there.  

In [109]:
def updateUs(ulist,alignedSeq):

    newUlist=list() 
    oldUcounter=0 #keeps position in the old U list
    alignCounter=0 #keeps position in the aligned sequence
    while alignCounter<len(alignedSeq):
        #if there is a dash, put in a zero at the corresponding position
        if alignedSeq[alignCounter]=="-":
            newUlist.append(0)
            alignCounter=alignCounter+1
        else:
            newUlist.append(ulist[oldUcounter])
            oldUcounter=oldUcounter+1
            alignCounter=alignCounter+1
            if (oldUcounter+1)==len(ulist):
                newUlist.append(ulist[oldUcounter])
    return newUlist

In [112]:
ND3updatedUs=list()
print ND3alignedShort[3].uList
for i in range(len(ND3alignedShort)):
    ND3updatedUs.append(ND3alignedShort[i])
    ND3updatedUs[i].uList=updateUs(ND3alignedShort[i].uList,ND3alignedShort[i].noUs)
print ND3updatedUs[3].uList

[0, 0, 1, 0, 1, 0, 0, 0, 5, 0, 3, 0, 3, 2, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 0, 1, 1, 4, 4, 3, 2, 2, 1, 1, 3, 0, 5, 1, 1, 1, 3, 0, 2, 2, 2, 4, 2, 3, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 3, 1, 2, 1, 1, 3, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 5, 0, 3, 0, 3, 2, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 0, 1, 1, 4, 4, 3, 2, 2, 1, 1, 3, 0, 5, 1, 1, 1, 3, 0, 2, 2, 2, 4, 2, 3, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 3, 1, 2, 1, 1, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# masterUlistmaker function
### This function creates a master U list that contains the highest number of Us observed at every edited position.  It makes a dummy list of all of the different numbers of Us at a given position and then records the maximum from the dummy list, and then moves on the the next position.  

In [110]:
def masterUlistmaker(seqList):
    #make a master list of most abundant Us
    masterUList=list()

    # make a list containing all of the numbers of Us at position i
    for i in range(len(seqList[0].uList)):
        dummyUlist=list()
        for j in range(len(seqList)):
            dummyUlist.append(seqList[j].uList[i])
        #record the highest number in the master list
        masterUList.append(max(dummyUlist))
    return masterUList


In [113]:
ND3masterU=masterUlistmaker(ND3updatedUs)
print ND3masterU

[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 5, 0, 3, 0, 3, 2, 1, 0, 2, 5, 1, 3, 5, 1, 2, 2, 1, 0, 3, 4, 4, 3, 1, 0, 0, 9, 1, 6, 1, 6, 2, 0, 0, 7, 0, 2, 4, 0, 2, 2, 3, 1, 0, 4, 0, 1, 1, 0, 3, 1, 1, 0, 1, 1, 0, 0, 2, 0, 1, 0, 3, 1, 1, 4, 1, 0, 0, 2, 0, 0, 3, 1, 4, 2, 0, 2, 3, 6, 1, 2, 2, 2, 2, 2, 0, 2, 2, 4, 4, 5, 2, 3, 5, 5, 5, 1, 5, 2, 2, 5, 5, 2, 3, 5, 7, 7, 5, 4, 5, 6, 3, 3, 3, 2, 3, 3, 4, 2, 3, 1, 2, 1, 1, 5, 0, 0, 0, 1, 0, 0, 0, 1, 1, 3, 3, 5, 0, 1, 1, 0, 4, 4, 3, 1, 0, 1, 6, 2, 1, 2, 1, 1, 0, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0]


# putTheUsBack function
### This function reinserts the correct number of Us back into the noUs sequence, and it inserts dashes if the number of Us at a given position in a sequence is less that the highest number observed at that position

In [114]:
def putTheUsBack(align1,newulist1,masterUlist):
    
    newalign1=""

    for i in range(len(align1)):
        #insert the number of Us specified if it is the same or greater than those specified in the masterUlist
        if newulist1[i]>=masterUlist[i]:
            newalign1=newalign1+"U"*newulist1[i]
        #insert dashes to fill space
        else:
            newalign1=newalign1+"-"*(masterUlist[i]-newulist1[i])+"U"*newulist1[i]
        newalign1=newalign1+align1[i]
    #This checks the last position as the uList is always one item longer than the alignment string
    if newulist1[len(align1)]>=masterUlist[len(align1)]:
        newalign1=newalign1+"U"*newulist1[len(align1)]
    else:
        newalign1=newalign1+"-"*(masterUlist[len(align1)]-newulist1[len(align1)])+"U"*newulist1[len(align1)]

    return newalign1

In [115]:
ND3shortFinal=list()
for i in range(len(ND3updatedUs)):
    ND3shortFinal.append(ND3updatedUs[i])
    ND3shortFinal[i].alignedSeq=putTheUsBack(ND3shortFinal[i].noUs,ND3shortFinal[i].uList,ND3masterU)
    print ND3shortFinal[i].alignedSeq

UCAAAAAAUCCUCGCCUUUUUACUUUAGUUUGUUAUCAUUAUUUUUAUAUUUGUUUUUG-A-UAUUGUGGUUUA--UUAUUUUAUUUAUAGGUUUUUUUUUAUGUUUUUUAUGUUUUUUAUUGCAUUUUUUUGAUUGUUUUCGUUGUUGUUUGUGGUUUUCGUGUGGUUUGUAUGAUAUGAAUUCA-CGUUUG-GUGUUUUAUACAUUGGAUUUAUGUUUUGUUAGUUGUUUGUUUUUUGUAUUGUUA--A--AUUCC--AUUA-UUUG---UG-UUUUGUUGUUUGUUUUUG----UG-----AUA-----G-UGUUG-UUUUAUUUUUGUUA--UG-----G-UUUUUUG--UUUUUG----UG----GUUUUUGUUUUUUG-UUG--UA--UG-UA--UA---G----G--AUUUGUG-UG-GUAUUUUUGGGAUCACGUAUAUUUG--UG----UGGUGUAAUUUUAUUUUGUUUAUGAUGUUUUUUGUUGUAUUAUA-CAUAUUAUAUUAAUAAAUAUAUAAAA
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------UUUUUG--UUUUUG-------G--UUUG--UUG-UUUUG---UUUGUUUG---AUUUG-UAUUUAUUUG--UUG--AUUUGUGUUGUGUA--UUUGGGAUCACGUAUA-UUGUUUGUUUUUGGUG-AA-UUU

# lineFit function
### This function rewrites the aligned sequences in a more visually appealing fashion within a single string.  It takes the final aligned sequences and records a certain number of characters per line, with their titles shown at the end of each line.  It also prints a title supplied by the user.

In [116]:
def lineFit(seqList,alignmentName,lineMax):
    alignLength=len(seqList[0].alignedSeq)
    alignment=">"+alignmentName+"\n"
    start=0 #counter that shows where to start copying for each new line
    #copy down a set number of characters from each alignment
    #repeat until what is left is shorter than the specified max line length
    while alignLength>lineMax:
        for j in range(len(seqList)):
            for i in range(start,(start+lineMax)):
                alignment=alignment+seqList[j].alignedSeq[i]
            alignment=alignment+"-"+seqList[j].title+"\n"
        alignment=alignment+"\n"
        start=start+lineMax
        alignLength=alignLength-lineMax
    #copy down the remaining characters from each alignment
    for j in range(len(seqList)):
        for i in range(start,(start+alignLength)):
            alignment=alignment+seqList[j].alignedSeq[i]
        alignment=alignment+"-"+seqList[j].title+"\n"

    return alignment

In [118]:
ND3shortAlignment=lineFit(ND3shortFinal,"ND3 short alignment",50)
print ND3shortAlignment

>ND3 short alignment
UCAAAAAAUCCUCGCCUUUUUACUUUAGUUUGUUAUCAUUAUUUUUAUAU-ND3Conv
---------------------------------------------------ND3AltF01
------AAUCCUCGCCUUUUUACUUUAGUUUGUUAUCA--A-----A-A--ND3AltA02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUAUCA--A-----A-A--ND3Alt4E01
---------------------------------------------------ND3Alt2A03
------AAUCCUCGCC-UUUUACUUUAGUUUGUUAUCA--A-----G-A--ND3Alt2C02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUAUCA--A-----A-G--ND3AltD02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUAUCA--A-----A-A--ND3AltE02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUAUCA--A-----A-G--ND3Alt3B01
------AAUCCUCGCCUUUUUACUUUAGUUUGUUAUCA--A-----A-A--ND3Alt3B02

UUGUUUUUG-A-UAUUGUGGUUUA--UUAUUUUAUUUAUAGGUUUUUUUU-ND3Conv
---------------------------------------------------ND3AltF01
--G----UGUAUUA--G-GG---AUUUUA----A---A-AGG---------ND3AltA02
--G----UGUAUUA--G-GG---AUUUUA----A---A-AGG---------ND3Alt4E01
---------------------------------------------------ND3Alt2A03
--G----UGUAUUA--G-GG---AUUUUA----A---A-AGG---------ND3Alt2C0

# multipleEdSeqAlign function
### This function ties all of the previous functions together.  It extracts the sequences from a file using extractSequences, then it aligns the sequences using alignAll. Then it updates their u lists using updateUs and it generates a master u list using masterUlistmaker.  Finally, it reinserts the Us into the aligned sequences using putTheUsBack and creates an alignment using lineFit.

In [119]:
def multipleEdSeqAlign(filename,alignmentName,lineMax):
    allSeq=extractSequences(filename)
    updatedSeq=alignAll(allSeq)
    for i in range(len(updatedSeq)):
        updatedSeq[i].uList=updateUs(updatedSeq[i].uList,updatedSeq[i].noUs)      
    masterUlist=masterUlistmaker(updatedSeq)
    for i in range(len(updatedSeq)):
        updatedSeq[i].alignedSeq=putTheUsBack(updatedSeq[i].noUs,updatedSeq[i].uList,masterUlist)
    alignment=lineFit(updatedSeq,alignmentName,lineMax)
    return alignment

In [123]:
import time
start=time.time()
print multipleEdSeqAlign("ND3align3.fa","ND3alternativealignment",80)
end=time.time()
print (end-start)

>ND3alternativealignment
UCAAAAAAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCAUUA-UUUUUAUAUUUGUUUUUG-A----UAUUG--UG--GU-ND3ConvED
UCAAAAAAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-A---G----UGUA---UUA--G---G--G--ND3Uned
------AAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-A---G----UGUA---UUA--G---G--G--ND3AltA02
---------------------------------------------------------------------------------ND3Alt2A03
------AAUCCUCGCC-UUUUACUUUAGUUUGUUA-UCA--A------G-A---G----UGUA---UUA--G---G--G--ND3AltC02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-G---G----UGUA---UUA--G---G--G--ND3AltD02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-A---G----UGUA---UUA--G---G--G--ND3AltE02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-G---G----UGUA---UUA--G---G--G--ND3Alt3B01
------AAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-A---G----UGUA---UUA--G---G--G--ND3Alt3B02
------AAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-A---G----UGUA---UUA--G---G--G--ND3Alt3D01
------AAUCCUCGCCUUUUUACUUUAGUUUGUUA-UCA--A------A-A---G----UGUA

In [122]:
import time
start=time.time()
print multipleEdSeqAlign("CR3allMarkandJames.fa","CR3alternativealignment",80)
end=time.time()
print (end-start)

>CR3alternativealignment
AGAAAUAUAAAUAUGUGUAUGAUAUAUAAAAACAAUGUUUGA----UUG-UUUG----GUUUUG----UUGUUUUUUUA--CR3Conv001
---------------------------------------------------------------------------------CR3AltB01
---------------------------------------------------------------------------------CR3AltG01
---------------------------------------------------------------------------------CR3AltH01
---------------------------------------------------------------------------------CR3AltA02
---------------------------------------------------------------------------------CR3AltB02
---------------------------------------------------------------------------------CR3AltD01
---------------------------------------------------------------------------------CR3AltC01
---------------------------------------------------------------------------------CR3AltE01
-GAAAUAUAAAUAUGUGUAUGAUAUAUAAAAACAA-G---GA----UUG--UUGUUUUGUUUUGUUUUUUA--------U-CR3Alt1D01
AGAAAUAUAAAUAUGUGUAUGAUAUAUAAAAACAAUG---GA-----UG----G----G----

# Conclusion
### I was able to create a tool that aligned sequences with different numbers of uridines at different positions. It takes 30-45 seconds to run on a group of 20-30 sequences, however this time seems to increase exponentially.  This does not seem like a good tool to use on transcriptomics data obtained from deep sequencing, however, if converted to a more powerful language, this could be possible.  
### The score numbers for this tool were difficult to identify.  It seems that giving the gap penalty a greater weight than the mismatch penalty achieves the desired results.  
